In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
from pprint import pprint
from sys import maxsize
from itertools import permutations
# import mlrose
# import numpy as np
# import six
# import sys
# sys.modules['sklearn.externals.six'] = six
# import mlrose
import random

In [ ]:
random.seed(41)

In [ ]:
data = pd.read_csv("../input/latitude-and-longitude-for-every-country-and-state/world_country_and_usa_states_latitude_and_longitude_values.csv")


In [ ]:
#This dataset contain country_codes along with their geo locations

In [ ]:
data = data[['country_code','latitude','longitude']]

In [ ]:
# data.dropna(axis=0,inplace=True,)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
data.head()

In [ ]:
# data.reset_index()

In [ ]:
data.nunique()

In [ ]:
data.shape

In [ ]:
#Generating a random sample to assign as delivery guys 
#These 10 indexes will be treated as the location of delivery guys
del_guys = random.sample(range(243), 10)
del_guys

In [ ]:
del_guys_data = data.iloc[del_guys].reset_index()

In [ ]:
del_guys_data.head()

In [ ]:
data.drop(data.index[del_guys],inplace=True)
data.reset_index(drop=True, inplace=True)
data.head()

In [ ]:
data.shape

In [ ]:
#Creating a matrix that stores the location of every delivery guy to all other locations
glob_matrix = {}
for i in range(len(del_guys_data)):
    cord1 = (del_guys_data['latitude'][i],del_guys_data['longitude'][i])
    
    val_list = []
    for j in range(len(data)):
        cord2 = (data['latitude'][j],data['longitude'][j])
        if cord1 == cord2:
            loc_dist = (data['country_code'][j],0)
            val_list.append(loc_dist)
            continue
            
        else:
            res = geopy.distance.geodesic(cord1, cord2).km
            loc_dist = (data['country_code'][j],round(res))
            val_list.append(loc_dist)
    
    glob_matrix[del_guys_data['country_code'][i]] = val_list

#this matrix will be helpful in assigning new orders based on minimum distance
print(glob_matrix['HU']) #showing the distances of HU to every other location

In [ ]:
#Now choosing 30 random order location from the dataset
del_location = random.sample(range(len(data)), 30)
del_location_df = data.iloc[del_location].reset_index()
del_location_df.head()

In [ ]:
#Now we have the dataframe of delivery guys and delivery locations,We need to calculate the distance of every delivery guy
#from every delivery location and the one with the lowest distance will get the order. This Information is already
#present in the glob_matrix
assigned_order = {}

for i in range(len(del_location_df)):
    low = 1000000
    val_list = []
    val_tup = ()
    for key,val in glob_matrix.items():
        for v in val:
            if v[0] == del_location_df['country_code'][i]:
                curr_dist = v[1]
#                 val_list.append((v[0],v[1]))
                if curr_dist <= low:
                    low = curr_dist
                    val_tup = (key,v[0],low)
                
#                 print(key,v[0],del_location_df['country_code'][i],low,curr_dist,i,val_tup)

    if val_tup[0] not in assigned_order.keys():
        assigned_order[val_tup[0]] = []
        assigned_order[val_tup[0]].append((val_tup[0],0))
        assigned_order[val_tup[0]].append((val_tup[1],val_tup[2]))
    else:
        assigned_order[val_tup[0]].append((val_tup[1],val_tup[2]))

#This dictionary determines the who gets the order to deliver
pprint(assigned_order)
            

In [ ]:
#we need this dataframe to calculate the fina TSP matrix
new_data = pd.read_csv("../input/latitude-and-longitude-for-every-country-and-state/world_country_and_usa_states_latitude_and_longitude_values.csv")

In [ ]:
# V = 10 #number of deliveries
def travellingSalesmanProblem(graph, s,V):
 
    # store all vertex apart from source vertex
    vertex = []
    hist_dict = {}
    for i in range(V):
        if i != s:
            vertex.append(i)
 
    min_path = maxsize
    next_permutation=permutations(vertex)
    
    for i in next_permutation:
        path_list = []
        current_pathweight = 0
        
        k = s
        for j in i:
            current_pathweight += graph[k][j]
            path_list.append(str(k)+str(j)) #storing the path taken
            k = j
            
        current_pathweight += graph[k][s]
        path_list.append(str(k)+str(s))
        hist_dict[current_pathweight] = path_list
        
 
        # update minimum
        min_path = min(min_path, current_pathweight)
         
    return min_path,hist_dict
 
 

               
#Here s is the location of the delivery guy (starting point)
# s=0
# cost,data_weight = travellingSalesmanProblem(matrix_tsp, s)

# print(cost)
# print(data_weight[cost])
# location_list = []
# for code in data_weight[cost]:
#     location_list.append(location_code[int(code[0])])

# print(location_list) #Displaying the location that need to be visited optimally

In [ ]:
#Now using this dictionary we need to create a matrix for every delivery guy to apply TSP
matrix = []
matrix_key = {}
for key,val in assigned_order.items():
    location_code = []
    location_list = []
    for v1 in val:
        val_list = []
        cord1 = new_data[new_data['country_code'] == v1[0]]
        location_code.append(v1[0])
        point1 = (cord1['latitude'].values[0],cord1['longitude'].values[0])
        for v2 in val:
            cord2 = new_data[new_data['country_code'] == v2[0]]
            point2 = (cord2['latitude'].values[0],cord2['longitude'].values[0])
            if v1[0] == v2[0]:
                val_list.append(0)
            
            else:
                res = geopy.distance.geodesic(point1, point2).km
                val_list.append(round(res))
        matrix.append(val_list) #matrix corresponding to assigned delivery guy
    print(location_code) #these are the initial sequence of locaitons
#     matrix.append(-1)
    cost,data_weight = travellingSalesmanProblem(matrix, 0,len(val)) #Here 0 is the location where we need to start from i,e delivery guy's location
    print(data_weight[cost]) #this list contains the indexes that need to be visited first
    
    #This loop is mapping these indexes to the locations
    for code in data_weight[cost]:
        location_list.append(location_code[int(code[0])])
    print(f"{location_list[0]}'s order sequence {location_list}") #this list is the final order of the location that need to be visited
    print()
      

BELOW I USED A DIFFERENT LIBRARY TO PERFORM OPTIMISATION

In [ ]:
# matrix = []
# for i in range(len(samp)):
#     cord1 = (samp['latitude'][i],samp['longitude'][i])
#     val = ()
#     val_list = []
#     for j in range(i,len(samp)):
        
#         cord2 = (samp['latitude'][j],samp['longitude'][j])
#         if cord1 == cord2:
#             val_list.append(0)
#             continue
            
#         else:
#             res = geopy.distance.geodesic(cord1, cord2).km
#             val = (i,j,round(res))
#             val_list.append(round(res))
#             matrix.append(val)
    
#     matrix_tsp.append(val_list)

# print(matrix)

In [ ]:
# matrix_tsp = []

# #Creating a matrix of distances between every location
# #Note: Same locations have 0 distance in the matrix
# for i in range(len(samp)):
#     cord1 = (samp['latitude'][i],samp['longitude'][i])
    
#     val_list = []
#     for j in range(len(samp)):
        
#         cord2 = (samp['latitude'][j],samp['longitude'][j])
# #         print(samp['country_code'][i],cord1,cord2)
#         if cord1 == cord2:
#             val_list.append(0)
#             continue
            
#         else:
#             res = geopy.distance.geodesic(cord1, cord2).km
# #             val = (i,j,round(res))
#             val_list.append(round(res))
    
#     matrix_tsp.append(val_list)

# location_code = list(samp['country_code']) #Storing the lcoation in the list
# pprint(matrix_tsp)
# print(location_code)

In [ ]:
# fitness_dists = mlrose.TravellingSales(distances = matrix)
# problem_fit = mlrose.TSPOpt(length = 10, fitness_fn = fitness_dists,maximize=False)
# best_state, best_fitness = mlrose.genetic_alg(problem_fit, random_state = 2)
# print('The best state found is: ', best_state)
# print('The fitness at the best state is: ', best_fitness)

In [ ]:
# V = 10 #number of deliveries
# def travellingSalesmanProblem(graph, s):
 
#     # store all vertex apart from source vertex
#     vertex = []
#     hist_dict = {}
#     for i in range(V):
#         if i != s:
#             vertex.append(i)
 
#     min_path = maxsize
#     next_permutation=permutations(vertex)
    
#     for i in next_permutation:
#         path_list = []
#         current_pathweight = 0
        
#         k = s
#         for j in i:
#             current_pathweight += graph[k][j]
#             path_list.append(str(k)+str(j)) #storing the path taken
#             k = j
            
#         current_pathweight += graph[k][s]
#         path_list.append(str(k)+str(s))
#         hist_dict[current_pathweight] = path_list
        
 
#         # update minimum
#         min_path = min(min_path, current_pathweight)
         
#     return min_path,hist_dict
 
 

               
# #Here s is the location of the delivery guy (starting point)
# s=0
# cost,data_weight = travellingSalesmanProblem(matrix_tsp, s)

# print(cost)
# print(data_weight[cost])
# location_list = []
# for code in data_weight[cost]:
#     location_list.append(location_code[int(code[0])])

# print(location_list) #Displaying the location that need to be visited optimally